In [1]:
file = "test_data_sensor_1.csv"


In [4]:
file.split("_")[-1].split(".")[0]

'1'

In [2]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import numpy as np
import os

def store_predictions_in_mongodb(sensor_id, dates, predictions):
    try:
        # logger.info("calling DB configuration")
        db = os.getenv("db")
        host = os.getenv("host")
        port = os.getenv("port")
        collection_name = os.getenv("collection1")

        MONGO_URL = f"mongodb://{host}:{port}"

        labeled_to_original_mapping = {
            0: "5f718c439c7a78.65267835",
            1: "62a9920f75c931.62399458",
        }

        client = MongoClient(MONGO_URL)
        db1 = client[db]
        collection = db1[collection_name]

        for date in set(dates.date):
            date_str = date.strftime('%Y-%m-%d')
            document_id = f"{sensor_id}_{date_str}"

            data = {
                "_id": document_id,
                "sensor_id": labeled_to_original_mapping.get(sensor_id, sensor_id),
                "creation_time": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "millisecond": int(datetime.now().timestamp() * 1000),
                "data": {}
            }

            # Filter predictions for the current date
            date_predictions = predictions[dates.date == date]

            # Populate the 'data' dictionary with hourly predictions
            for i, row in date_predictions.iterrows():
                prediction_float = round(float(row['predictions']), 4)
                data["data"][str(i)] = {
                    "pre_kwh": prediction_float,
                    "pre_current": 0.0,
                    "pre_load": 0.0,
                    "act_kwh": 0.0,
                    "act_load": 0.0
                }

            data_dict = {key: float(value) if isinstance(value, (float, np.integer, float, np.floating)) else value
                         for key, value in data.items()}

            # Insert data into MongoDB
            collection.insert_one(data_dict)

        client.close()

    except Exception as e:
        print(e)

# Assuming 'dates' and 'predictions' are your pandas DataFrame columns
dates = pd.to_datetime(['2023-11-28', '2023-11-29', '2023-11-30'])
predictions = pd.DataFrame({
    'date': dates.repeat(3),  # Repeat each date three times for demonstration
    'predictions': np.random.rand(len(dates) * 3) * 1000  # Random predictions
})

# Call the function with your sensor_id
store_predictions_in_mongodb(sensor_id=0, dates=dates, predictions=predictions)


Port must be an integer between 0 and 65535: 'None'


In [1]:
import datetime


In [7]:
today = datetime.date.today()
first = today.replace(day=1)

last_month = first - datetime.timedelta(days=1)
print(last_month.strftime("%Y-%m"))

2023-11


In [10]:
from datetime import timedelta
import datetime

end_date = datetime.today().replace(day=1)  # Start of the current month
start_date = (end_date - timedelta(days=1)).replace(day=1)  # Start of the last month

print(end_date)
print(start_date)
# last_month_data = df[(df['creation_time'] >= start_date) & (df['creation_time'] < end_date)]
# 
# # Sort data by 'creation_time' in ascending order
# last_month_data = last_month_data.sort_values(by='creation_time', ascending=True)
# 
# # Convert DataFrame to a list of dictionaries (MongoDB-friendly format)
# data_to_insert = last_month_data.to_dict(orient='records')

AttributeError: module 'datetime' has no attribute 'today'